# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [43]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [44]:
test = pd.read_csv('flights_test.csv')

In [45]:
df = pd.read_csv('flights_merged.csv', index_col=0)

In [46]:
df = df.dropna(axis=0, subset=['crs_elapsed_time'])

In [47]:
df = df.sample(frac=0.1)

In [48]:
df

fl_date mkt_unique_carrier branded_code_share mkt_carrier  \
133834  2018-01-04                 WN                 WN          WN   
10516   2019-01-01                 AA                 AA          AA   
153084  2019-01-05                 WN                 WN          WN   
21278   2019-01-07                 AA                 AA          AA   
74337   2019-01-02                 DL                 DL          DL   
...            ...                ...                ...         ...   
194671  2018-01-03                 F9                 F9          F9   
135268  2018-01-04                 WN                 WN          WN   
35071   2018-01-04                 AA       AA_CODESHARE          AA   
74085   2019-01-06                 DL                 DL          DL   
31863   2018-01-03                 AA                 AA          AA   

        mkt_carrier_fl_num op_unique_carrier tail_num  op_carrier_fl_num  \
133834                5970                WN   N8317M               5970   
10516                 2052                AA   N927UW               2052   
153084                4315                WN   N8705Q               4315   
21278                 2061                AA   N755US               2061   
74337                 1817                DL   N854DN               1817   
...                    ...               ...      ...                ...   
194671                1123                F9   N312FR               1123   
135268                1913                WN   N793SA               1913   
35071                 5981                YV   N920FJ               5981   
74085                 1601                DL   N992AT               1601   
31863                 2692                AA   N9630A               2692   

        origin_airport_id origin  ... distance    passengers     capacity  \
133834              10257    ALB  ...   1073.0   2642.250000   556.000000   
10516               13930    ORD  ...    599.0  13383.333333  4325.333333   
153084              12892    LAX  ...   1750.0  15628.333333  1233.333333   
21278               15016    STL  ...    550.0  11667.500000  3890.500000   
74337               12889    LAS  ...    368.0   3652.500000   832.166667   
...                   ...    ...  ...      ...           ...          ...   
194671              11193    CVG  ...   1678.0   1349.000000    99.000000   
135268              10397    ATL  ...    696.0   9297.000000  2028.500000   
35071               11298    DFW  ...    569.0   6701.500000  1653.500000   
74085               13198    MCI  ...    393.0   4464.000000  1622.500000   
31863               13930    ORD  ...    719.0   4775.000000  2553.000000   

       total_gallons  dep_delay  nas_delay  security_delay  \
133834  1.723195e+08  13.464286   7.125000             0.0   
10516   3.079852e+08   5.817460   5.440000             0.0   
153084  1.723195e+08   7.289157  10.250000             0.0   
21278   3.079852e+08   4.142857   9.600000             0.0   
74337   3.087060e+08   4.370787  11.181818             0.0   
...              ...        ...        ...             ...   
194671  2.353042e+07   5.750000   0.500000             0.0   
135268  1.723195e+08  10.626667   2.800000             0.0   
35071   0.000000e+00  13.273585  11.785714             0.0   
74085   3.087060e+08  -4.740741  61.000000             0.0   
31863   3.079852e+08  20.682540  13.625000             0.0   

        late_aircraft_delay  weather_delay  carrier_delay  
133834            19.875000       0.000000      19.250000  
10516              8.160000      10.200000       4.040000  
153084            11.312500       0.000000      12.437500  
21278             12.950000       7.600000       7.600000  
74337             31.272727       0.000000      10.909091  
...                     ...            ...            ...  
194671            42.500000       0.000000       3.000000  
135268            15.933333       0.000000      12.600000  
35071             15.82

In [49]:
y= df['arr_delay']

In [50]:
X = df

In [51]:
X = X.drop(columns=['arr_delay'])

In [52]:
X.fl_date = X.fl_date.apply(pd.to_datetime)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [53]:
X = X[['origin_airport_id', 'dest_airport_id', 'distance', 'passengers', 'crs_elapsed_time','capacity', 'total_gallons', 'dep_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay',
       'weather_delay', 'carrier_delay']]
#dropped flight date :( 

In [54]:
og_X = X

In [55]:
scaler = StandardScaler()

In [56]:
X = pd.DataFrame(scaler.fit_transform(X))

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8)


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [215]:
#data_dmatrix = xgb.DMatrix(data=X,label=y, enable_categorical=True)

In [216]:
#objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 7, alpha = 0.1,

In [16]:
lr = LinearRegression()

In [17]:
lr.fit(X_train, y_train) 

LinearRegression()

In [18]:
lr.score(X_train, y_train)

0.159267945741012

In [19]:
lr.score(X_test, y_test)

0.17962858113333258

### Random Forest

RandomForestRegressor()

In [20]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100)
 

In [21]:
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor()

In [22]:
y_pred = regressor.predict(X_test)

In [23]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 29.54027670901269
Mean Squared Error: 1643.2005992991765
Root Mean Squared Error: 40.53641078461654


In [24]:
feature_importance= pd.Series(regressor.feature_importances_, index= x_label).sort_values(ascending=False)
feature_importance

dep_delay              0.220665
crs_elapsed_time       0.182979
nas_delay              0.090361
late_aircraft_delay    0.068881
carrier_delay          0.065717
passengers             0.063864
capacity               0.061855
origin_airport_id      0.059212
dest_airport_id        0.058391
distance               0.049645
total_gallons          0.042902
weather_delay          0.029439
security_delay         0.006089
dtype: float64

In [25]:
params = {'n_estimators' : [100, 150, 200, 300], 'max_depth': [3, 10, 20, 30]}

In [26]:
grid = GridSearchCV(regressor, params)

In [27]:
grid_result = grid.fit(X_train, y_train)

In [28]:
print(grid_result.best_score_)
print( grid_result.best_params_['n_estimators'])
print( grid_result.best_params_['max_depth'])

0.11898671987936302
100
3


### XGBOOST

In [29]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 7, alpha = 0.1,)

In [ ]:
xg_reg.fit(X_train,y_train)


In [ ]:
preds = xg_reg.predict(X_test)

In [ ]:
xg_reg.score(X_train,y_train)

In [ ]:
xg_reg.score(X_test,y_test)

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.